<a href="https://colab.research.google.com/github/Nick088Official/GFPGAN-Fix/blob/master/GFPGAN_fix_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GFPGAN Inference Demo  (Restore faces of images)

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2101.04061)
[![GitHub Stars](https://img.shields.io/github/stars/TencentARC/GFPGAN?style=social)](https://github.com/TencentARC/GFPGAN)
[![download](https://img.shields.io/github/downloads/TencentARC/GFPGAN/total.svg)](https://github.com/TencentARC/GFPGAN/releases)

## GFPGAN - Towards Real-World Blind Face Restoration with Generative Facial Prior

GFPGAN is a blind face restoration algorithm towards real-world face images. <br>
It leverages the generative face prior in a pre-trained GAN (*e.g.*, StyleGAN2) to restore realistic faces while precerving fidelity. <br>

**Limitations**: GFPGAN could not handle all the low-quality faces in the real world. Therefore, it may fail on your own cases.

###Enjoy! :-)

<img src="https://xinntao.github.io/projects/GFPGAN_src/gfpgan_teaser.jpg" width="800">


In [ ]:
#@title Installation
#@markdown Before start, make sure that you choose
#@markdown * Runtime Type = Python 3
#@markdown * Hardware Accelerator = GPU

#@markdown in the **Runtime** menu -> **Change runtime type**

#@markdown By running this, we clone the repository, set up the envrironment, and download the pre-trained model which we use 1.2 , 1.3 , 1.4 , RestoreFormer , CodeFormer.

# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN-Fix
!git clone https://github.com/Nick088Official/GFPGAN-Fix.git
%cd GFPGAN-Fix

import torch
from IPython.display import clear_output
import numpy as np
import glob
from os.path import isfile, join
import os
from google.colab import files
import shutil

if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

# Set up the environment
# We install and use new-BasicSR for both training and inference
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
!pip uninstall -y basicsr # have to uninstall it as it gives error to GFPGAN but still gets installed from other package
!pip install -r requirements.txt # which installs other packages such as new-basicsr

if device == "cuda":
  !BASICSR_EXT=True pip install new-basicsr # for installing cuda extensions for v1 model for colorization

# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.1.0/GFPGANv1.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/RestoreFormer.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/CodeFormer.pth -P experiments/pretrained_models
clear_output()
print("Installed GFPGAN, it's requirements and the models.")

In [ ]:
#@title Upload Input Image/Video

input_type = "video" #@param ['image', 'video']

if input_type == "image":
  upload_folder = 'inputs/upload'
else:
  upload_folder = 'videos'
  import subprocess
  cap = cv2.VideoCapture(upload_folder)
  fps = cap.get(cv2.CAP_PROP_FPS)
  cap.release()


if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)

# upload input
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  if input_type == "image":
    print(f'moved {filename} to {dst_path}')
  else:
    print(f'moved {filename} of {fps} fps to {dst_path}')
  shutil.move(filename, dst_path)

## 3. Inference
Usage: `python inference_gfpgan.py -i inputs/whole_imgs -o results [options]...`

Options:
```
  -v version           GFPGAN model version. Option: 1 | 1.2 | 1.3 | 1.4. | RestoreFormer
  -s upscale           The final upsampling scale of the image. Default: 2
  -bg_upsampler        background upsampler. Default: realesrgan
  -bg_tile             Tile size for background sampler, 0 for no tile during testing. Default: 400
  -suffix              Suffix of the restored faces
  -only_center_face    Only restore the center face
  -aligned             Input are aligned faces
  -ext                 Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto
```

In [ ]:
#@title Inference
# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions
# You can find the different models in https://github.com/TencentARC/GFPGAN#european_castle-model-zoo

if input_type == "video":
  # assign directory
  directory = 'videos' #PATH_WITH_INPUT_VIDEOS
  zee = 0
  def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    size2 = (0,0)

    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        size2 = size
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size2)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()


for filename in os.listdir(directory):

    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):


      print("PROCESSING :"+str(f)+"\n")
      # Read the video from specified path

      #video to frames
      cam = cv2.VideoCapture(str(f))

      try:

          # PATH TO STORE VIDEO FRAMES
          if not os.path.exists('upload'):
              os.makedirs('upload')

      # if not created then raise error
      except OSError:
          print ('Error: Creating directory of data')

      # frame
      currentframe = 0

      #clear all folders


      #deletes upscaled frames from previous video
      #for f in os.listdir(result_folder):
      #  os.remove(os.path.join(result_folder, f))




      while(True):

          # reading from frame
          ret,frame = cam.read()

          if ret:
              # if video is still left continue creating images
              name = 'upload/frame' + str(currentframe) + '.jpg'

              # writing the extracted images
              cv2.imwrite(name, frame)


                # increasing counter so that it will
                # show how many frames are created
              currentframe += 1
              print(currentframe)
          else:
              #deletes all the videos you uploaded for upscaling
              #for f in os.listdir(video_folder):
              #  os.remove(os.path.join(video_folder, f))

              break

        # Release all space and windows once done
      cam.release()
      cv2.destroyAllWindows()

      #apply super-resolution on all frames of a video
      #scale factor is by 3.5x

!rm -rf results

options = "-v 1.3 -s 2 --bg_upsampler realesrgan" #@param {type:"string"}

!python inference_gfpgan.py -i inputs/upload -o results {options}

if input_type == "video":
  #convert super res frames to .avi
  pathIn = 'results/restored_imgs/'

  zee = zee+1
  fName = "video"+str(zee)
  filenameVid = f"{fName}.avi"

  pathOut = "results_videos/"+filenameVid


  convert_frames_to_video(pathIn, pathOut, fps)

  #after processing frames converted to .avi video , delete upscaled frames from previous video
  for f in os.listdir('results/restored_imgs'):
      os.remove(os.path.join('results/restored_imgs', f))

  #convert .avi to .mp4
  src = 'results_videos/'
  dst = 'results_mp4_videos/'

  for root, dirs, filenames in os.walk(src, topdown=False):
      #print(filenames)
      for filename in filenames:
          print('[INFO] 1',filename)
          try:
              _format = ''
              if ".flv" in filename.lower():
                  _format=".flv"
              if ".mp4" in filename.lower():
                  _format=".mp4"
              if ".avi" in filename.lower():
                  _format=".avi"
              if ".mov" in filename.lower():
                  _format=".mov"

              inputfile = os.path.join(root, filename)
              print('[INFO] 1',inputfile)
              outputfile = os.path.join(dst, filename.lower().replace(_format, ".mp4"))
              subprocess.call(['ffmpeg', '-i', inputfile, outputfile])
          except:
              print("An exception occurred")

!ls results/cmp

In [ ]:
#@title Download results
#@markdown It will download you a .zip, unzip it however you want
#@markdown - The 'cmp' folder contains a comparison between the Input vs Output faces,
#@markdown - The 'cropped_faces' folder contains only the cropped Input Face
#@markdown - The 'restored_faces' folder contains only the Output Face (Restored one)
#@markdown - The 'restored_imgs' folder contains the whole Output Image restored with also the Background

# download the result
!ls results
print('Download results')
os.system('zip -r download.zip results')
files.download("download.zip")